### This part is about how to get the prediction from the account and text and testing the running time of the whole process.
#### By: Yue Yang(UIN: 625008110)

Our web app is built on python 3. Because of the difference between python 2.x and python 3.x, if we want to use the model created by the teammate, we must use the same versin python. So, we need check the version at first.

In [17]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/Users/letheyue/anaconda2/envs/yy_py3/bin/python
3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:14:23) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
sys.version_info(major=3, minor=6, micro=5, releaselevel='final', serial=0)


In [19]:
import twitter
import time

api = twitter.Api(consumer_key=CONSUMER_KEY,
                     consumer_secret=CONSUMER_SECRET,
                     access_token_key=ACCESS_TOKEN_KEY,
                     access_token_secret=ACCESS_TOKEN_SECRET)

### Step 1: test the account model

#### step 1.1: get the account feature

In [20]:
import pandas as pd
import numpy as np

def get_user(user_id=None, screen_name=None):
    json = api.GetUser(user_id=user_id, screen_name=screen_name, include_entities=True, return_json=False)
    json_data = json._json
    
    # feature Count of favorite tweets
    Count_of_favorite_tweets = int(json_data['favourites_count'])
    # feature Friends to follower ratio
    if json_data['followers_count'] == 0:
        Friends_to_follower_ratio = float(100000)
    else:
        Friends_to_follower_ratio = (float(json_data['friends_count']) / json_data['followers_count'])
    # feature Total status count
    Total_status_count = int(json_data['statuses_count'])
    
    # feature Default profile image & Default profile
    # def_p_na','def_p_false','def_p_true'
    if json_data['default_profile_image'] == 'FALSE':
        Def_p_img_false = 1.0
        Def_p_img_true = 0.0
        Def_p_img_na = 0.0
    elif json_data['default_profile_image'] == 'True':
        Def_p_img_false = 0.0
        Def_p_img_true = 1.0
        Def_p_img_na = 0.0
    else:
        Def_p_img_false = 0.0
        Def_p_img_true = 0.0
        Def_p_img_na = 1.0

    if json_data['default_profile'] == 'FALSE':
        Def_p_false = 1.0
        Def_p_true = 0.0
        Def_p_na = 0.0
    elif json_data['default_profile'] == 'True':
        Def_p_false = 0.0
        Def_p_true = 1.0
        Def_p_na = 0.0
    else:
        Def_p_false = 0.0
        Def_p_true = 0.0
        Def_p_na = 1.0
    
    # feature Account ages
    created_at = json_data['created_at']
    Account_age = survival_time(created_at)

    # feature User name and screen_name
    User_name = json_data['name']
    Screen_name = json_data['screen_name']
    User_name_digit, User_name_char = counter(User_name)
    Screen_name_digit, Screen_name_char = counter(Screen_name)

    # feature Length of description and Description text
    description_pre = json_data['description']
    Description_length = len(description_pre)
    
    # feature Average tweet per day
    # Average_tweets_per_day = Total_status_count / float(Account_age)
    
    # feature listed count
    Listed_count = json_data['listed_count']
    
    # feature of description, the number of hashtag, @ and url
    Bio_hashtag = json_data['description'].count('#')
    Bio_at = json_data['description'].count('@')
    Bio_url = json_data['description'].count('http')

    feature = pd.DataFrame(index=[0])
    feature['favorite_count'] = Count_of_favorite_tweets
    feature['friends_to_followers'] = Friends_to_follower_ratio
    feature['statuses_count'] = Total_status_count
    feature['def_p_img_na'] = Def_p_img_na
    feature['def_p_img_false'] = Def_p_img_false
    feature['def_p_img_true'] = Def_p_img_true
    feature['def_p_na'] = Def_p_na
    feature['def_p_false'] = Def_p_false
    feature['def_p_true'] = Def_p_true
    feature['age'] = Account_age
    feature['name_letter'] = User_name_char
    feature['name_num'] = User_name_digit
    feature['screen_letter'] = Screen_name_char
    feature['screen_num'] = Screen_name_digit
    feature['des_len'] = Description_length
#     feature['avg_tweet_per_day'] = Average_tweets_per_day
#     feature['des_text'] = Description_tfidf
    feature['listed_count'] = Listed_count
    feature['bio_hashtag'] = Bio_hashtag
    feature['bio_at'] = Bio_at
    feature['bio_url'] = Bio_url

    return feature

In [21]:
def survival_time(created_at):
    # get the account ages: crawl at time - created at time 
    current_time = time.localtime(time.time())
    current_year = current_time.tm_year
    current_month = current_time.tm_mon
    current_day = current_time.tm_mday

    meta = created_at.split(" ")
    created_month = meta[1]
    if created_month == 'Jan':
        created_month = int(1)
    elif created_month == 'Feb':
        created_month = int(2)
    elif created_month == 'Mar':
        created_month = int(3)
    elif created_month == 'Apr':
        created_month = int(4)
    elif created_month == 'May':
        created_month = int(5)
    elif created_month == 'Jun':
        created_month = int(6)
    elif created_month == 'Jul':
        created_month = int(7)
    elif created_month == 'Aug':
        created_month = int(8)
    elif created_month == 'Sep':
        created_month = int(9)
    elif created_month == 'Oct':
        created_month = int(10)
    elif created_month == 'Nov':
        created_month = int(11)
    elif created_month == 'Dec':
        created_month = int(12)
    created_day = int(meta[2])
    created_year = int(meta[5])

    Account_age = (current_year - created_year) * 365 + (current_month - created_month) * 30 + (current_day - created_day)

    return Account_age

def counter(name):
    # counter of char & counter of digit
    numbers = sum(c.isdigit() for c in name)
    words   = sum(c.isalpha() for c in name)

    return numbers, words

### measure the running time of the whole process of prediction through account model

In [22]:
from sklearn.externals import joblib

def measure_time(user_id=None, screen_name=None):
    # measure the running time of getting the features from the input
    start = time.clock()
    
    with open("classifier/rf_user_2.pkl", "rb") as file_handler:
        loaded_pickle = joblib.load(file_handler)
    feature = get_user(screen_name=screen_name)
    np_feature = np.asarray((feature))
    pred_account = loaded_pickle.predict(np_feature.tolist())
    print(pred_account[0])
    
    end = time.clock()
    print('function took %0.5f ms' % ((end-start)*1000.0))

In [23]:
measure_time(screen_name='realDonaldTrump')

0
function took 1749.29100 ms


### Step 2: test the text model

#### Step 2.1 : get the latest tweet of the user

In [24]:
timeline = api.GetUserTimeline(screen_name="realDonaldTrump")

In [25]:
timeline

[Status(ID=986643179452747776, ScreenName=realDonaldTrump, Created=Wed Apr 18 16:30:39 +0000 2018, Text='Best wishes to Prime Minister @Netanyahu and all of the people of Israel on the 70th Anniversary of your Great Inde… https://t.co/xRwM6h8nfl'),
 Status(ID=986576502358671361, ScreenName=realDonaldTrump, Created=Wed Apr 18 12:05:42 +0000 2018, Text='Slippery James Comey, the worst FBI Director in history, was not fired because of the phony Russia investigation wh… https://t.co/iJO4tejdw6'),
 Status(ID=986555533535731712, ScreenName=realDonaldTrump, Created=Wed Apr 18 10:42:23 +0000 2018, Text='Mike Pompeo met with Kim Jong Un in North Korea last week. Meeting went very smoothly and a good relationship was f… https://t.co/5zMrs47vF3'),
 Status(ID=986547093610299392, ScreenName=realDonaldTrump, Created=Wed Apr 18 10:08:50 +0000 2018, Text='A sketch years later about a nonexistent man. A total con job, playing the Fake News Media for Fools (but they know… https://t.co/EcZ0Hs5IJg'),
 Sta

In [26]:
list_text = list()
index = 0
for item in timeline:
    index += 1
    data = item._json
    list_text.append(data['text'])
    if index == 1:
        break

In [27]:
print("The latest tweet of the user is:")
list_text

The latest tweet of the user is:


['Best wishes to Prime Minister @Netanyahu and all of the people of Israel on the 70th Anniversary of your Great Inde… https://t.co/xRwM6h8nfl']

#### step2 : pre-process the text

In [28]:
import re
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r"(?P<url>https?://[^\s]+)"
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner_updated(text):
    try:
        mention = re.search(pat1, text).group()
    except:
        mention = ''
    try:
        url = re.search(pat2, text).group("url")
        o = urlparse(url)
        netloc = o.netloc
        path = p.path
    except:
        netloc = ''
        path = ''
    soup = BeautifulSoup(text, 'html.parser')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\u2026", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    lower_case = stripped.lower()
    letters_only = re.sub("[^a-zA-Z]", " ", lower_case)
    letters_only = letters_only + netloc + path + mention
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [29]:
text = tweet_cleaner_updated(list_text[0])

In [30]:
print("The text of pre-processing is:")
text

The text of pre-processing is:


'best wishes to prime minister and all of the people of israel on the th anniversary of your great inde Netanyahu'

#### step 3: convert the text to vector features

In [32]:
import pickle
import numpy as np

with open('classifier/pos_hmean.p', 'rb') as fp:
    w2v_pos_hmean_01 = pickle.load(fp, encoding='latin1')

In [33]:
def get_w2v_general(tweet, size, vectors):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tweet.split():
        try:
            vec += vectors[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [34]:
text = get_w2v_general(text, 200, w2v_pos_hmean_01)

In [35]:
print("The vector after converting we get:")
text

The vector after converting we get:


array([[ 0.29363613, -0.13414309,  0.23163194, -0.19407212, -0.08049656,
        -0.4527868 ,  0.57960978,  0.8215894 ,  0.34427389,  0.22781383,
         0.06217574,  0.59507235,  0.04497319, -0.29434509,  0.66448316,
        -0.83753664, -0.49833031, -0.00834269,  0.78028165,  0.6007587 ,
        -0.25693855,  0.37303438,  0.83198129, -0.25038035, -0.3521909 ,
        -0.2967161 ,  0.26776673, -0.27395859, -0.67257661,  0.11864048,
        -0.17793136, -0.62188897,  0.47119962, -0.85762493,  0.06408137,
        -0.23893477, -0.35725511,  0.1270298 ,  0.23592222,  0.29180517,
         1.05172305, -0.3015131 , -0.00380763,  0.18064061,  0.32413879,
        -0.09178447,  0.2558675 ,  0.26000569, -0.39713907, -0.17771211,
        -0.49688988,  1.40566771,  0.12925726, -0.59132192, -0.08977496,
         0.48567726,  0.14237436,  0.01197417,  1.29863416,  0.6431513 ,
        -0.06247757, -0.48087426, -0.64353769,  0.06104737, -0.57256479,
         0.27234417,  0.05765121,  0.10007235, -0.2

####  step 4: get the prediction

In [37]:
from keras.models import load_model
loaded_w2v_model = load_model('classifier/w2v_01_best_weights.10-0.9346.hdf5')

In [38]:
pred = loaded_w2v_model.predict(text)

In [39]:
pred[0][0]

1.0

### measure the running time of the whole process of prediction through text model

In [46]:
def measure_running_time(user_id=None, screen_name=None):
    # measure the running time of getting the features from the input
    start = time.clock()
    # get the timeline of user
    timeline = api.GetUserTimeline(screen_name=screen_name)
    
    # get the first text
    list_text = list()
    index = 0
    for item in timeline:
        index += 1
        data = item._json
        list_text.append(data['text'])
        if index == 1:
            break
            
    # pre-processing the data
    text = tweet_cleaner_updated(list_text[0])
    
    # load the function
    with open('classifier/pos_hmean.p', 'rb') as fp:
        w2v_pos_hmean_01 = pickle.load(fp, encoding='latin1')
    loaded_w2v_model = load_model('classifier/w2v_01_best_weights.10-0.9346.hdf5')
    
    # get the prediction value
    text = get_w2v_general(text, 200, w2v_pos_hmean_01)
    pred = loaded_w2v_model.predict(text)
    print(pred[0][0])
    
    end = time.clock()
    print('function took %0.5f ms' % ((end-start)*1000.0))

In [47]:
measure_running_time(screen_name='realDonaldTrump')

1.0
function took 2559.46000 ms
